In [1]:
# Import pandas for basic data analysis
import pandas as pd
# Import sqlite3 to work with .sqlite databases
import sqlite3

In [9]:
# Establish a connection to the database
conn = sqlite3.connect('database.sqlite')

# Load each table as a pandas data frame
matches = pd.read_sql("SELECT * FROM Match", conn)
teams = pd.read_sql("SELECT * FROM Team", conn)
team_attr = pd.read_sql("SELECT * FROM Team_Attributes", conn)

In [10]:
# LEFT JOIN team_api_id, team_long_name ON home_team_api_id = team_api_id
matches = matches.merge(teams[['team_api_id', 'team_long_name']], 
                        left_on='home_team_api_id',
                        right_on='team_api_id',
                        how='left')
# Rename team_long_name to home_team_name and drop team_api_id
matches = matches.rename(columns={'team_long_name': 'home_team_name'}).drop(columns=['team_api_id'])

# LEFT JOIN team_api_id, team_long_name ON away_team_api_id = team_api_id
matches = matches.merge(teams[['team_api_id', 'team_long_name']], 
                        left_on='away_team_api_id',
                        right_on='team_api_id',
                        how='left')
# Rename team_long_name to away_team_name and drop team_api_id
matches = matches.rename(columns={'team_long_name': 'away_team_name'}).drop(columns=['team_api_id'])

In [11]:
# Convert dates to datetime objects
matches['date'] = pd.to_datetime(matches['date'], format='%Y-%m-%d %H:%M:%S')
team_attr['date'] = pd.to_datetime(team_attr['date'], format='%Y-%m-%d %H:%M:%S')

In [12]:
# Sort team_attr for merging
team_attr = team_attr.sort_values(['team_api_id', 'date'])

In [13]:
# home_attr gets team_attr renamed to have home_{col_name}
home_attr = team_attr.rename(columns=lambda x: f"home_{x}" if x not in ['id', 'team_api_id', 'date'] else x)
# away_attr gets team_attr renamed to have away_{col_name}
away_attr = team_attr.rename(columns=lambda x: f"away_{x}" if x not in ['id', 'team_api_id', 'date'] else x)

In [14]:
# Join matches and home_attr on date where home_team_api_id matches team_api_id
matches = pd.merge_asof(matches.sort_values('date'),
                        home_attr.sort_values('date'),
                        left_on='date',
                        right_on='date',
                        left_by='home_team_api_id',
                        right_by='team_api_id',
                        direction='backward')

# Join matches and home_attr on date where home_team_api_id matches team_api_id
matches = pd.merge_asof(matches.sort_values('date'),
                        away_attr.sort_values('date'),
                        left_on='date',
                        right_on='date',
                        left_by='away_team_api_id',
                        right_by='team_api_id',
                        direction='backward')

In [15]:
# Create a results column that classifies a match as either a win or a loss
matches['result'] = matches.apply(lambda x: 'HomeWin' if x['home_team_goal'] > x['away_team_goal'] else ('AwayWin' if x['home_team_goal'] < x['away_team_goal'] else 'Draw'), axis=1)

In [16]:
# Save the dataset to be used later
matches.to_csv('dataset.csv', index=False)